### Load data

In [1]:
# Load packages

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zipfile
import os

In [2]:

import pandas as pd
import geopandas as gpd

# Read CSV
df = pd.read_csv('./data2/cd_2018.csv')

df.fillna(0, inplace=True)

df.head()

# df['CD116FP'].type()


,CD116FP,Voter Turnout Percentage,18-29,30-44,45-64,65 and older,Women,In Poverty,Did not finish high school,Bachelors or more,White,Black,Asian,Hispanic,urbanization_pct
0,1,57.441623,21.3,23.5,34.8,20.4,51.5,11.7,9.3,32.6,75.3,20.300000,1.8,1.7,41.747327
1,7,56.067900,18.3,22.8,34.7,24.1,51.9,10.1,11.9,19.8,94.1,3.514391,0.5,1.3,14.610002
2,2,57.572041,19.7,24.2,34.7,21.3,52.1,11.9,9.3,30.2,88.1,8.800000,1.0,1.2,13.408745
3,14,62.144496,16.5,21.0,36.6,25.9,51.9,6.8,8.1,34.7,92.1,4.811186,1.5,2.1,14.156915
4,6,54.016244,17.0,21.2,36.3,25.4,50.3,14.9,11.2,15.8,95.7,2.189251,0.0,0.8,10.754026


In [3]:
# Function for cleaning data
def clean_cd_data(file_path, rename_dict, columns):
    #Read file
    data = pd.read_csv(file_path,
                          header=2,
                          skiprows=[3,4])
    #Rename columns
    data = data.rename(columns=rename_dict)

    #Filter out non-Ohio data
    data = data[data['State name'] == 'Ohio']

    #Specify columns
    data = data[columns]

    #Add column for join
    data['CD116FP'] = data['Congressional district'].astype(int)

    return data

In [4]:
# Get new voter turnout data

turnout_file = "./data2/voter_turnout_2018.csv"

turnout_dict = {'Voting rate3': 'Voting rate'}

turnout_columns = ['State name', 'Congressional district', 'Voting rate']

voter_turnout = clean_cd_data(turnout_file, turnout_dict,turnout_columns)

voter_turnout.head()

,State name,Congressional district,Voting rate,CD116FP
298,Ohio,1,55.542510,1
299,Ohio,2,51.913271,2
300,Ohio,3,43.973845,3
301,Ohio,4,47.415238,4
302,Ohio,5,51.644570,5


In [5]:
df = df.merge(
    voter_turnout[['CD116FP', 'Voting rate']],
    on='CD116FP',
    how='left'
)

df['18-44'] = df['18-29'] + df['30-44']

df.head(20)

print(df.columns)

Index(['CD116FP', 'Voter Turnout Percentage', '18-29', '30-44', '45-64',
       '65 and older', 'Women', 'In Poverty', 'Did not finish high school',
       'Bachelors or more', 'White', 'Black', 'Asian', 'Hispanic',
       'urbanization_pct', 'Voting rate', '18-44'],
      dtype='object')


In [7]:
# Select columns

df = df[['CD116FP', 'Voter Turnout Percentage', '18-44', '45-64',
       '65 and older', 'Women', 'In Poverty', 'Did not finish high school',
       'Bachelors or more', 'White', 'Black', 'Asian', 'Hispanic',
       'urbanization_pct']]

df.loc[:, df.columns != 'Voter Data'] = df.loc[:, df.columns != 'Voter Data'].round(1)

df.head()

,CD116FP,Voter Turnout Percentage,18-44,45-64,65 and older,Women,In Poverty,Did not finish high school,Bachelors or more,White,Black,Asian,Hispanic,urbanization_pct
0,1,57.4,44.8,34.8,20.4,51.5,11.7,9.3,32.6,75.3,20.3,1.8,1.7,41.7
1,7,56.1,41.1,34.7,24.1,51.9,10.1,11.9,19.8,94.1,3.5,0.5,1.3,14.6
2,2,57.6,43.9,34.7,21.3,52.1,11.9,9.3,30.2,88.1,8.8,1.0,1.2,13.4
3,14,62.1,37.5,36.6,25.9,51.9,6.8,8.1,34.7,92.1,4.8,1.5,2.1,14.2
4,6,54.0,38.2,36.3,25.4,50.3,14.9,11.2,15.8,95.7,2.2,0.0,0.8,10.8


In [ ]:
# Export as csv

df.to_csv('./data2/cd_2018_cleaned.csv', index=False)